In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import json
import re
import pandas as pd
from joblib import Parallel, delayed

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
def add_ad_to_car_dict(ad):
    car_url = ad
    response = requests.get(car_url)
    response.encoding = 'utf-8'
    page = BeautifulSoup(response.text, 'html.parser')
    try:
        json_data = json.loads(
            page.find('script', type="application/ld+json").string)
    except:
        json_data = None
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')
        catalog_url = catalog_url.get('href')
    except:
        return
    response_catalog = requests.get(catalog_url)
    response_catalog.encoding = 'utf-8'
    page_catalog = BeautifulSoup(response_catalog.text, 'html.parser')
    try:
        json_data_catalog = json.loads(page_catalog.find(
            'script', type="application/json", id='initial-state').string)
    except:
        json_data_catalog = None
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        json_data_equip = None
    try:
        json_data['description'] = json_data['description'].replace('\n', ' ')
        json_data['description'] = re.sub('\W+', ' ', json_data['description'])
    except:
        json_data['description'] = ''
    try:
        options_dict = json_data_catalog['state']['compare']['selected'][0]['options']
        options_list = [key for key in list(
            options_dict.keys()) if options_dict[key] == 1]
    except:
        options_list = []
    car_dict.update({ad: {}})
    try:
        car_dict[ad]['car_url'] = json_data['offers']['url']
    except:
        car_dict[ad]['car_url'] = None
    try:
        car_dict[ad]['bodyType'] = json_data['bodyType']
    except:
        car_dict[ad]['bodyType'] = None
    try:
        car_dict[ad]['brand'] = json_data['brand']
    except:
        car_dict[ad]['brand'] = None
    try:
        car_dict[ad]['color'] = json_data['color']
    except:
        car_dict[ad]['color'] = None
    try:
        car_dict[ad]['complectation_dict'] = options_list
    except:
        car_dict[ad]['complectation_dict'] = None
    try:
        car_dict[ad]['description'] = json_data['description']
    except:
        car_dict[ad]['description'] = None
    try:
        car_dict[ad]['engineDisplacement'] = json_data['vehicleEngine']['engineDisplacement']
    except:
        car_dict[ad]['engineDisplacement'] = None
    try:
        car_dict[ad]['enginePower'] = json_data['vehicleEngine']['enginePower']
    except:
        car_dict[ad]['enginePower'] = None
    try:
        car_dict[ad]['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        car_dict[ad]['equipment_dict'] = None
    try:
        car_dict[ad]['fuelType'] = json_data['fuelType']
    except:
        car_dict[ad]['fuelType'] = None
    try:
        car_dict[ad]['image'] = json_data['image']
    except:
        car_dict[ad]['image'] = None
    try:
        car_dict[ad]['mileage'] = page.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['mileage'] = None
    try:
        car_dict[ad]['modelDate'] = json_data['modelDate']
    except:
        car_dict[ad]['modelDate'] = None
    try:
        car_dict[ad]['model_name'] = page.find_all(
            'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        car_dict[ad]['model_name'] = None
    try:
        car_dict[ad]['name'] = json_data['name']
    except:
        car_dict[ad]['name'] = None
    try:
        car_dict[ad]['numberOfDoors'] = json_data['numberOfDoors']
    except:
        car_dict[ad]['numberOfDoors'] = None
    try:
        car_dict[ad]['parsing_unixtime'] = int(time.time())
    except:
        car_dict[ad]['parsing_unixtime'] = None
    try:
        car_dict[ad]['price'] = json_data['offers']['price']
    except:
        car_dict[ad]['price'] = None
    try:
        car_dict[ad]['priceCurrency'] = json_data['offers']['priceCurrency']
    except:
        car_dict[ad]['priceCurrency'] = None
    try:
        car_dict[ad]['productionDate'] = json_data['productionDate']
    except:
        car_dict[ad]['productionDate'] = None
    try:
        car_dict[ad]['sell_id'] = page.find(
            'div', class_='CardHead__infoItem CardHead__id').text[2:]
    except:
        car_dict[ad]['sell_id'] = None
    try:
        car_dict[ad]['views'] = page.find(
            'div', class_='CardHead__infoItem CardHead__views').text.split()[0]
    except:
        car_dict[ad]['views'] = None
    try:
        car_dict[ad]['date_added'] = page.find(
            'div', class_='CardHead__infoItem CardHead__creationDate').text
    except:
        car_dict[ad]['date_added'] = None
    try:
        car_dict[ad]['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        car_dict[ad]['super_gen'] = None
    try:
        car_dict[ad]['vehicleConfiguration'] = json_data['vehicleConfiguration']
    except:
        car_dict[ad]['vehicleConfiguration'] = None
    try:
        car_dict[ad]['vehicleTransmission'] = json_data['vehicleTransmission']
    except:
        car_dict[ad]['vehicleTransmission'] = None
    try:
        car_dict[ad]['Владельцы'] = page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['Владельцы'] = None
    try:
        car_dict[ad]['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        car_dict[ad]['Владение'] = None
    try:
        car_dict[ad]['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        car_dict[ad]['ПТС'] = None
    try:
        car_dict[ad]['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        car_dict[ad]['Привод'] = None
    try:
        car_dict[ad]['Руль'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        car_dict[ad]['Руль'] = None
    try:
        car_dict[ad]['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        car_dict[ad]['Состояние'] = None
    try:
        car_dict[ad]['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        car_dict[ad]['Таможня'] = None
    try:
        car_dict[ad]['region'] = page.find('div', class_='CardBreadcrumbs').find_all(
            'div', class_='CardBreadcrumbs__item')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['region'] = None

df_ads = pd.read_csv('Project_6.csv', sep=',', header=None)
all_ads = df_ads[0].tolist()
car_dict = {}
i = 125000
for ad in all_ads[125000:150000]:
    print(i)
    add_ad_to_car_dict(ad)
    i += 1
    if i % 10 == 0:
        print(f'Completed {i} of total {len(all_ads)}')
    else:
        continue
df_6 = pd.DataFrame.from_dict(car_dict, orient='index')
df_6.to_csv('Final_df_6.csv')

125000
125001
125002
125003
125004
125005
125006
125007
125008
125009
Completed 125010 of total 269381
125010
125011
125012
125013
125014
125015
125016
125017
125018
125019
Completed 125020 of total 269381
125020
125021
125022
125023
125024
125025
125026
125027
125028
125029
Completed 125030 of total 269381
125030
125031
125032
125033
125034
125035
125036
125037
125038
125039
Completed 125040 of total 269381
125040
125041
125042
125043
125044
125045
125046
125047
125048
125049
Completed 125050 of total 269381
125050
125051
125052
125053
125054
125055
125056
125057
125058
125059
Completed 125060 of total 269381
125060
125061
125062
125063
125064
125065
125066
125067
125068
125069
Completed 125070 of total 269381
125070
125071
125072
125073
125074
125075
125076
125077
125078
125079
Completed 125080 of total 269381
125080
125081
125082
125083
125084
125085
125086
125087
125088
125089
Completed 125090 of total 269381
125090
125091
125092
125093
125094
125095
125096
125097
125098
125099
Com

125798
125799
Completed 125800 of total 269381
125800
125801
125802
125803
125804
125805
125806
125807
125808
125809
Completed 125810 of total 269381
125810
125811
125812
125813
125814
125815
125816
125817
125818
125819
Completed 125820 of total 269381
125820
125821
125822
125823
125824
125825
125826
125827
125828
125829
Completed 125830 of total 269381
125830
125831
125832
125833
125834
125835
125836
125837
125838
125839
Completed 125840 of total 269381
125840
125841
125842
125843
125844
125845
125846
125847
125848
125849
Completed 125850 of total 269381
125850
125851
125852
125853
125854
125855
125856
125857
125858
125859
Completed 125860 of total 269381
125860
125861
125862
125863
125864
125865
125866
125867
125868
125869
Completed 125870 of total 269381
125870
125871
125872
125873
125874
125875
125876
125877
125878
125879
Completed 125880 of total 269381
125880
125881
125882
125883
125884
125885
125886
125887
125888
125889
Completed 125890 of total 269381
125890
125891
125892
12589

126592
126593
126594
126595
126596
126597
126598
126599
Completed 126600 of total 269381
126600
126601
126602
126603
126604
126605
126606
126607
126608
126609
Completed 126610 of total 269381
126610
126611
126612
126613
126614
126615
126616
126617
126618
126619
Completed 126620 of total 269381
126620
126621
126622
126623
126624
126625
126626
126627
126628
126629
Completed 126630 of total 269381
126630
126631
126632
126633
126634
126635
126636
126637
126638
126639
Completed 126640 of total 269381
126640
126641
126642
126643
126644
126645
126646
126647
126648
126649
Completed 126650 of total 269381
126650
126651
126652
126653
126654
126655
126656
126657
126658
126659
Completed 126660 of total 269381
126660
126661
126662
126663
126664
126665
126666
126667
126668
126669
Completed 126670 of total 269381
126670
126671
126672
126673
126674
126675
126676
126677
126678
126679
Completed 126680 of total 269381
126680
126681
126682
126683
126684
126685
126686
126687
126688
126689
Completed 126690 

Completed 127390 of total 269381
127390
127391
127392
127393
127394
127395
127396
127397
127398
127399
Completed 127400 of total 269381
127400
127401
127402
127403
127404
127405
127406
127407
127408
127409
Completed 127410 of total 269381
127410
127411
127412
127413
127414
127415
127416
127417
127418
127419
Completed 127420 of total 269381
127420
127421
127422
127423
127424
127425
127426
127427
127428
127429
Completed 127430 of total 269381
127430
127431
127432
127433
127434
127435
127436
127437
127438
127439
Completed 127440 of total 269381
127440
127441
127442
127443
127444
127445
127446
127447
127448
127449
Completed 127450 of total 269381
127450
127451
127452
127453
127454
127455
127456
127457
127458
127459
Completed 127460 of total 269381
127460
127461
127462
127463
127464
127465
127466
127467
127468
127469
Completed 127470 of total 269381
127470
127471
127472
127473
127474
127475
127476
127477
127478
127479
Completed 127480 of total 269381
127480
127481
127482
127483
127484
12748

128184
128185
128186
128187
128188
128189
Completed 128190 of total 269381
128190
128191
128192
128193
128194
128195
128196
128197
128198
128199
Completed 128200 of total 269381
128200
128201
128202
128203
128204
128205
128206
128207
128208
128209
Completed 128210 of total 269381
128210
128211
128212
128213
128214
128215
128216
128217
128218
128219
Completed 128220 of total 269381
128220
128221
128222
128223
128224
128225
128226
128227
128228
128229
Completed 128230 of total 269381
128230
128231
128232
128233
128234
128235
128236
128237
128238
128239
Completed 128240 of total 269381
128240
128241
128242
128243
128244
128245
128246
128247
128248
128249
Completed 128250 of total 269381
128250
128251
128252
128253
128254
128255
128256
128257
128258
128259
Completed 128260 of total 269381
128260
128261
128262
128263
128264
128265
128266
128267
128268
128269
Completed 128270 of total 269381
128270
128271
128272
128273
128274
128275
128276
128277
128278
128279
Completed 128280 of total 26938

...

146509
Completed 146510 of total 269381
146510
146511
146512
146513
146514
146515
146516
146517
146518
146519
Completed 146520 of total 269381
146520
146521
146522
146523
146524
146525
146526
146527
146528
146529
Completed 146530 of total 269381
146530
146531
146532
146533
146534
146535
146536
146537
146538
146539
Completed 146540 of total 269381
146540
146541
146542
146543
146544
146545
146546
146547
146548
146549
Completed 146550 of total 269381
146550
146551
146552
146553
146554
146555
146556
146557
146558
146559
Completed 146560 of total 269381
146560
146561
146562
146563
146564
146565
146566
146567
146568
146569
Completed 146570 of total 269381
146570
146571
146572
146573
146574
146575
146576
146577
146578
146579
Completed 146580 of total 269381
146580
146581
146582
146583
146584
146585
146586
146587
146588
146589
Completed 146590 of total 269381
146590
146591
146592
146593
146594
146595
146596
146597
146598
146599
Completed 146600 of total 269381
146600
146601
146602
146603
14660

147303
147304
147305
147306
147307
147308
147309
Completed 147310 of total 269381
147310
147311
147312
147313
147314
147315
147316
147317
147318
147319
Completed 147320 of total 269381
147320
147321
147322
147323
147324
147325
147326
147327
147328
147329
Completed 147330 of total 269381
147330
147331
147332
147333
147334
147335
147336
147337
147338
147339
Completed 147340 of total 269381
147340
147341
147342
147343
147344
147345
147346
147347
147348
147349
Completed 147350 of total 269381
147350
147351
147352
147353
147354
147355
147356
147357
147358
147359
Completed 147360 of total 269381
147360
147361
147362
147363
147364
147365
147366
147367
147368
147369
Completed 147370 of total 269381
147370
147371
147372
147373
147374
147375
147376
147377
147378
147379
Completed 147380 of total 269381
147380
147381
147382
147383
147384
147385
147386
147387
147388
147389
Completed 147390 of total 269381
147390
147391
147392
147393
147394
147395
147396
147397
147398
147399
Completed 147400 of tota

148101
148102
148103
148104
148105
148106
148107
148108
148109
Completed 148110 of total 269381
148110
148111
148112
148113
148114
148115
148116
148117
148118
148119
Completed 148120 of total 269381
148120
148121
148122
148123
148124
148125
148126
148127
148128
148129
Completed 148130 of total 269381
148130
148131
148132
148133
148134
148135
148136
148137
148138
148139
Completed 148140 of total 269381
148140
148141
148142
148143
148144
148145
148146
148147
148148
148149
Completed 148150 of total 269381
148150
148151
148152
148153
148154
148155
148156
148157
148158
148159
Completed 148160 of total 269381
148160
148161
148162
148163
148164
148165
148166
148167
148168
148169
Completed 148170 of total 269381
148170
148171
148172
148173
148174
148175
148176
148177
148178
148179
Completed 148180 of total 269381
148180
148181
148182
148183
148184
148185
148186
148187
148188
148189
Completed 148190 of total 269381
148190
148191
148192
148193
148194
148195
148196
148197
148198
148199
Completed 

148899
Completed 148900 of total 269381
148900
148901
148902
148903
148904
148905
148906
148907
148908
148909
Completed 148910 of total 269381
148910
148911
148912
148913
148914
148915
148916
148917
148918
148919
Completed 148920 of total 269381
148920
148921
148922
148923
148924
148925
148926
148927
148928
148929
Completed 148930 of total 269381
148930
148931
148932
148933
148934
148935
148936
148937
148938
148939
Completed 148940 of total 269381
148940
148941
148942
148943
148944
148945
148946
148947
148948
148949
Completed 148950 of total 269381
148950
148951
148952
148953
148954
148955
148956
148957
148958
148959
Completed 148960 of total 269381
148960
148961
148962
148963
148964
148965
148966
148967
148968
148969
Completed 148970 of total 269381
148970
148971
148972
148973
148974
148975
148976
148977
148978
148979
Completed 148980 of total 269381
148980
148981
148982
148983
148984
148985
148986
148987
148988
148989
Completed 148990 of total 269381
148990
148991
148992
148993
14899

149693
149694
149695
149696
149697
149698
149699
Completed 149700 of total 269381
149700
149701
149702
149703
149704
149705
149706
149707
149708
149709
Completed 149710 of total 269381
149710
149711
149712
149713
149714
149715
149716
149717
149718
149719
Completed 149720 of total 269381
149720
149721
149722
149723
149724
149725
149726
149727
149728
149729
Completed 149730 of total 269381
149730
149731
149732
149733
149734
149735
149736
149737
149738
149739
Completed 149740 of total 269381
149740
149741
149742
149743
149744
149745
149746
149747
149748
149749
Completed 149750 of total 269381
149750
149751
149752
149753
149754
149755
149756
149757
149758
149759
Completed 149760 of total 269381
149760
149761
149762
149763
149764
149765
149766
149767
149768
149769
Completed 149770 of total 269381
149770
149771
149772
149773
149774
149775
149776
149777
149778
149779
Completed 149780 of total 269381
149780
149781
149782
149783
149784
149785
149786
149787
149788
149789
Completed 149790 of tota